# OBIA Feature Extraction with Sentinel-2 Imagery


This notebook performs object-based feature extraction from Sentinel-2 imagery using the `nickyspatial` library.

Steps included:
1. Load Sentinel-2 bands (Red and NIR)
2. Compute NDVI
3. Segment the image using Quickshift
4. Compute zonal statistics (mean NDVI)
5. Extract geometry features (area, perimeter, compactness)
6. Compute GLCM texture features (contrast)
7. Save the results as GeoJSON and visualize
    

In [ ]:

import rasterio
import numpy as np
import geopandas as gpd
from nickyspatial.segmentation import quickshift_segmentation
from nickyspatial.zonal import zonal_stats_table
from skimage.feature import greycomatrix, greycoprops
from shapely.geometry import shape
import matplotlib.pyplot as plt


## Load Sentinel-2 Red and NIR bands

In [ ]:

# Update these paths with your actual file locations
red_path = "B04.tif"  # Red band (Band 4)
nir_path = "B08.tif"  # NIR band (Band 8)

with rasterio.open(red_path) as src:
    red = src.read(1).astype('float32')
    profile = src.profile
    transform = src.transform

with rasterio.open(nir_path) as src:
    nir = src.read(1).astype('float32')


## Compute NDVI

In [ ]:

ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)

plt.imshow(ndvi, cmap='RdYlGn')
plt.title("NDVI")
plt.colorbar()
plt.show()


## Segment NDVI image using Quickshift

In [ ]:

segments = quickshift_segmentation(ndvi, kernel_size=3, max_dist=6, ratio=0.5)

plt.imshow(segments, cmap='tab20')
plt.title("Segmented Image")
plt.colorbar()
plt.show()


## Compute Zonal Statistics (mean NDVI per object)

In [ ]:

features = {'mean_ndvi': ndvi}
gdf = zonal_stats_table(segments, features, transform=transform)


## Compute Texture Features (GLCM Contrast from NDVI)

In [ ]:

ndvi_int = ((ndvi + 1) * 127.5).astype('uint8')  # Scale to 0–255
glcm = greycomatrix(ndvi_int, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = greycoprops(glcm, 'contrast')[0, 0]
gdf['glcm_contrast'] = contrast  # Apply global contrast value (can be per object later)


## Add Geometry Features

In [ ]:

gdf['area'] = gdf.geometry.area
gdf['perimeter'] = gdf.geometry.length
gdf['compactness'] = (4 * np.pi * gdf['area']) / (gdf['perimeter'] ** 2)


## Save as GeoJSON

In [ ]:

gdf.to_file("obia_features.geojson", driver="GeoJSON")


## Visualize Results

In [ ]:

gdf.plot(column='mean_ndvi', cmap='YlGn', legend=True)
plt.title("Mean NDVI per Object")
plt.show()
